# Probando clustering the secuencias

### Importando datos

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename != 'sars_combined_sequences.csv':
            continue
        print(os.path.join(dirname, filename))
        df=pd.read_csv(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sarssequencedata/sars_combined_sequences.csv


### Vista de los datos

In [13]:
df

,sequence,id,description,label
0,ATATTAGGTTTTTACCTTCCCAGGTAACAAACCAACTAACTCTCGA...,MG772934.1,MG772934.1 Bat SARS-like coronavirus isolate b...,bat-SARS
1,ATATTAGGTTTTTACCTTCCCAGGTAACAAACCAACTAACTCTCGA...,MG772933.1,MG772933.1 Bat SARS-like coronavirus isolate b...,bat-SARS
2,ATATTAGGTTTTTACCTACCCAGGAAAAGCCAACCAACCTCGATCT...,KY417152.1,KY417152.1 Bat SARS-like coronavirus isolate R...,bat-SARS
3,ATATTAGGTTTTTACCTACCCAGGAAAAGCCAACCAACCTCGATCT...,KY417151.1,KY417151.1 Bat SARS-like coronavirus isolate R...,bat-SARS
4,ATATTAGGTTTTTACCTACCCAGGAAAAGCCAACCAACCTCGATCT...,KY417150.1,KY417150.1 Bat SARS-like coronavirus isolate R...,bat-SARS
...,...,...,...,...
449,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,MT253707.1,MT253707.1 Severe acute respiratory syndrome c...,SARS-cov-2
450,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,MT253708.1,MT253708.1 Severe acute respiratory syndrome c...,SARS-cov-2
451,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,MT253709.1,MT253709.1 Severe acute respiratory syndrome c...,SARS-cov-2
452,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,MT253710.1,MT253710.1 Severe acute respiratory syndrome c...,SARS-cov-2


In [33]:
train_set = df.head(100)
sequences = [i[:7] for i in train_set['sequence'].tolist()]
#sequences

In [34]:
def optimal_matching(seq1, seq2, insert_cost=1, delete_cost=1, substitution_cost=None):
    global n_transitions 
    global transitions_done 
    
    n = len(seq1)
    m = len(seq2)
    
    # Initialize the distance matrix
    dist_matrix = np.zeros((n + 1, m + 1))
    
    # Fill the first row and column with cumulative insertion and deletion costs
    for i in range(1, n + 1):
        dist_matrix[i, 0] = dist_matrix[i - 1, 0] + delete_cost
    for j in range(1, m + 1):
        dist_matrix[0, j] = dist_matrix[0, j - 1] + insert_cost
    
    # Fill the distance matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost_sub = substitution_cost(seq1[i - 1], seq2[j - 1]) if substitution_cost else (0 if seq1[i - 1] == seq2[j - 1] else 1)
            dist_matrix[i, j] = min(dist_matrix[i - 1, j] + delete_cost,  # Deletion
                                    dist_matrix[i, j - 1] + insert_cost,  # Insertion
                                    dist_matrix[i - 1, j - 1] + cost_sub)  # Substitution
    
    return dist_matrix[n, m]

In [35]:
n_transitions = 0
transitions_done = {}
# Custom substitution cost function

# TODO no esta hecho para que funcione aun, esta en proceso
def calculate_cost(a,b):
   
    if a == b:
        cost = 0
    else:
        n_transitions += 1
        if (a,b) in transitions_done.keys():
            transitions_done[(a,b)] += 1
        else:
            transitions_done[(a,b)] = 1
            
        p_ab = transitions_done[(a,b)] / n_transitions
        p_ba = transitions_done[(b,a)] / n_transitions if (b,a) in transitions_done.keys() else 0
            
        cost = 2 - p_ab - p_ba
    return cost

def calculate_cost2(a, b):
    """Cálculo del costo de sustitución entre dos elementos."""
    return 0 if a == b else 1

In [36]:
# Example usage
sequence1 = ['A', 'B', 'C', 'A', 'B']
sequence2 = ['A', 'C', 'B', 'A']

#substitucion_cost = create_substitution_cost_trate_method()

distance = optimal_matching(sequence1, sequence2, substitution_cost=calculate_cost2)
print(f"Optimal Matching Distance: {distance}")

Optimal Matching Distance: 3.0


In [ ]:
def generate_dissimilarities_matrix(sequences):
    n_sequences = len(sequences)
    matrix = np.zeros((n_sequences,n_sequences))

    for i in range(n_sequences):
        for j in range(n_sequences):
            if i == j:
                continue
            matrix[i,j] = optimal_matching(sequences[i],sequences[j], substitution_cost=calculate_cost2)

    return matrix

            
a=generate_dissimilarities_matrix(sequences)

In [39]:
a

array([[0., 0., 0., ..., 5., 5., 5.],
       [0., 0., 0., ..., 5., 5., 5.],
       [0., 0., 0., ..., 5., 5., 5.],
       ...,
       [5., 5., 5., ..., 0., 0., 0.],
       [5., 5., 5., ..., 0., 0., 0.],
       [5., 5., 5., ..., 0., 0., 0.]])

In [40]:
!pip install kmedoids 

ERROR: Could not find a version that satisfies the requirement kmedoids (from versions: none)
ERROR: No matching distribution found for kmedoids


In [41]:
from kmedoids import pam


pam_result = pam(a,3)


ModuleNotFoundError: No module named 'kmedoids'